In [2]:
import numpy as np
import tensorflow as tf
import random as rn
import os
import matplotlib.pyplot as plt
%matplotlib inline
os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(1)
rn.seed(1)
from keras import backend as K
#tf.compat.v1.set_random_seed(1)
#sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph())
#K.set_session(sess)
import sys 
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.layers import Conv1D,MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.optimizers import RMSprop
import keras.regularizers
import scipy
import math
import sys
import pandas as pd
from scipy.ndimage.filters import gaussian_filter1d
from sklearn.metrics import mean_squared_error
from scipy.stats import linregress
from scipy import interpolate
from scipy import signal
import pickle
import collections
from keras.callbacks import LearningRateScheduler
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TerminateOnNaN
from video_process_utils import *
import statsmodels.api as sm

In [3]:
target_col = 'GDI'

In [4]:
alldata_processed =\
    pd.read_csv("data/processed/alldata_processed_with_dev_residual.csv" )
alldata_processed['videoid'] = alldata_processed['videoid'].apply(lambda x: int(x))
alldata_processed['target_count'] = alldata_processed.groupby('videoid')[target_col].transform(lambda x: x.count())

In [5]:
alldata_processed = alldata_processed[alldata_processed[target_col].notnull()]
alldata_processed = alldata_processed[alldata_processed['target_count'] == 2]
ids_nonmissing_target = set(alldata_processed['videoid'].unique())

In [6]:
datasplit_df = pd.read_csv('./data/processed/train_test_valid_id_split.csv')
datasplit_df['videoid'] = datasplit_df['videoid'].apply(lambda x: int(x))
all_ids = set(datasplit_df['videoid']).intersection(ids_nonmissing_target)
train_ids = set(datasplit_df[datasplit_df['dataset'] == 'train']['videoid']).intersection(ids_nonmissing_target)
validation_ids = set(datasplit_df[datasplit_df['dataset'] == 'validation']['videoid']).intersection(ids_nonmissing_target)
test_ids = set(datasplit_df[datasplit_df['dataset'] == 'test']['videoid']).intersection(ids_nonmissing_target)

In [7]:
with open('./data/processed/all_processed_video_segments.pickle', 'rb') as handle:
    processed_video_segments = pickle.load(handle)

In [8]:
assert(np.sum(alldata_processed[target_col].isnull()) == 0)

In [9]:
x_columns_left = [2*LANK,2*LANK+1,2*LKNE,2*LKNE+1,
        2*LHIP,2*LHIP+1,2*LBTO,2*LBTO+1,50,52,54,56]
x_columns_right = [2*RANK,2*RANK+1,2*RKNE,2*RKNE+1,
        2*RHIP,2*RHIP+1,2*RBTO,2*RBTO+1,51,53,55,57]

target_dict_L = {}
target_dict_R = {}
for i in range(len(alldata_processed)):
    row = alldata_processed.iloc[i]
    if row['side'] == 'L':
        target_dict_L[row['videoid']] = row[target_col]
    if row['side'] == 'R':
        target_dict_R[row['videoid']] = row[target_col]

In [10]:
X = [t[2] for t in processed_video_segments if t[0] in all_ids]
X = np.stack(X)
X = np.concatenate([X[:,:,x_columns_left],X[:,:,x_columns_right]])

In [11]:
X_train = [t[2] for t in processed_video_segments if t[0] in train_ids]
X_train = np.stack(X_train)
X_train = np.concatenate([X_train[:,:,x_columns_left],X_train[:,:,x_columns_right]])

In [12]:
X_validation = [t[2] for t in processed_video_segments if t[0] in validation_ids]
X_validation = np.stack(X_validation)
X_validation = np.concatenate([X_validation[:,:,x_columns_left],X_validation[:,:,x_columns_right]])

In [13]:
y_train_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in train_ids]
y_train_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in train_ids]
y_train = np.array(y_train_L + y_train_R)

In [14]:
y_validation_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in validation_ids]
y_validation_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in validation_ids]
y_validation = np.array(y_validation_L + y_validation_R)

In [15]:
y_L = [target_dict_L[t[0]] for t in processed_video_segments if t[0] in all_ids]
y_R = [target_dict_R[t[0]] for t in processed_video_segments if t[0] in all_ids]
y = np.array(y_L + y_R)

In [16]:
videoid_count_dict = collections.Counter(np.array([t[0] for t in processed_video_segments]))

In [17]:
train_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in train_ids]
train_videoid_weights = train_videoid_weights + train_videoid_weights
train_videoid_weights = np.array(train_videoid_weights).reshape(-1,1)
validation_videoid_weights = [1./videoid_count_dict[t[0]] for t in processed_video_segments if t[0] in validation_ids]
validation_videoid_weights = validation_videoid_weights + validation_videoid_weights
validation_videoid_weights = np.array(validation_videoid_weights).reshape(-1,1)

In [18]:
target_min = np.min(y_train,axis=0)
target_range = np.max(y_train,axis=0) - np.min(y_train,axis=0)

In [19]:
y_train_scaled = ((y_train-target_min)/target_range).reshape(-1,1)
y_validation_scaled = ((y_validation-target_min)/target_range).reshape(-1,1)
y_validation_scaled = np.hstack([y_validation_scaled,validation_videoid_weights])
y_train_scaled = np.hstack([y_train_scaled,train_videoid_weights])
#c_i_factor is just a constant initially introduced to make the loss function more interpretable, but
#is probably not necessary if the model were to be retrained
c_i_factor = np.mean(np.vstack([train_videoid_weights,validation_videoid_weights])) 

In [20]:
vid_length = 124

In [21]:
def step_decay(initial_lrate,epochs_drop,drop_factor):
    def step_decay_fcn(epoch):
        return initial_lrate * math.pow(drop_factor, math.floor((1+epoch)/epochs_drop))
    return step_decay_fcn

epochs_drop,drop_factor = (10,0.8)
initial_lrate = 0.001
dropout_amount = 0.5
last_layer_dim = 10
filter_length = 8
conv_dim = 32
l2_lambda = 10**(-3.5)

def w_mse(weights):
    def loss(y_true, y_pred):
        return K.mean(K.sum(K.square(y_true-y_pred)*weights,axis=1)*tf.reshape(y_true[:,-1],(-1,1)))/c_i_factor
    return loss

#we don't want to optimize for the column counting video occurences, but
#they are included in the target so we can use that column for the loss function
weights = [1.0,0]

#fixed epoch budget of 100 that empirically seems to be sufficient 
n_epochs = 100

mse_opt = w_mse(weights)

#this is just for monitoring
mse_metric = w_mse(target_range**2*np.array(weights))

K.clear_session()

model = Sequential()
model.add(Conv1D(conv_dim,filter_length, input_dim=X_train.shape[2],input_length=vid_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(dropout_amount))
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv1D(conv_dim,filter_length,padding='same',kernel_regularizer=keras.regularizers.l2(l2_lambda)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=3))
model.add(Dropout(dropout_amount))
model.add(Flatten())
model.add(Dense(last_layer_dim,activation='relu'))
model.add(Dense(2, activation='linear'))

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 8, padding="same", input_shape=(124, 12))`


In [22]:
checkpoint_folder = "./data/checkpoints/cnn_checkpoints_%s" % (target_col)

In [23]:
train_model = True

if not os.path.exists(checkpoint_folder):
    os.makedirs(checkpoint_folder)

filepath=checkpoint_folder+"/weights-{epoch:02d}-{val_loss_1:.4f}.hdf5"
if train_model:

    opt = RMSprop(lr=0.0,rho=0.9, epsilon=1e-08, decay=0.0)
    model.compile(loss=mse_opt,metrics=[mse_metric],
                  optimizer=opt)


    checkpoint = \
        ModelCheckpoint(filepath, monitor='val_loss_1', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    lr = LearningRateScheduler(step_decay(initial_lrate,epochs_drop,drop_factor))

    history = model.fit(X_train, y_train_scaled,callbacks=[checkpoint,lr,TerminateOnNaN()],
              validation_data=(X_validation,y_validation_scaled),
              batch_size=32, epochs=n_epochs,shuffle=True)

Train on 40852 samples, validate on 4938 samples
Epoch 1/100
40852/40852 [==============================] - 75s 2ms/step - loss: 0.0958 - loss_1: 446.1005 - val_loss: 0.0254 - val_loss_1: 111.6367

Epoch 00001: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-01-111.6367.hdf5
Epoch 2/100
40852/40852 [==============================] - 68s 2ms/step - loss: 0.0184 - loss_1: 100.7041 - val_loss: 0.0153 - val_loss_1: 90.2141

Epoch 00002: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-02-90.2141.hdf5
Epoch 3/100
40852/40852 [==============================] - 72s 2ms/step - loss: 0.0160 - loss_1: 95.7544 - val_loss: 0.0157 - val_loss_1: 94.6918

Epoch 00003: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-03-94.6918.hdf5
Epoch 4/100
40852/40852 [==============================] - 105s 3ms/step - loss: 0.0151 - loss_1: 91.7415 - val_loss: 0.0157 - val_loss_1: 96.3010

Epoch 00004: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-04-96.

40852/40852 [==============================] - 67s 2ms/step - loss: 0.0112 - loss_1: 71.5283 - val_loss: 0.0114 - val_loss_1: 72.4519

Epoch 00035: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-35-72.4519.hdf5
Epoch 36/100
40852/40852 [==============================] - 58s 1ms/step - loss: 0.0112 - loss_1: 71.3054 - val_loss: 0.0109 - val_loss_1: 69.6521

Epoch 00036: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-36-69.6521.hdf5
Epoch 37/100
40852/40852 [==============================] - 53s 1ms/step - loss: 0.0112 - loss_1: 71.4255 - val_loss: 0.0122 - val_loss_1: 78.1076

Epoch 00037: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-37-78.1076.hdf5
Epoch 38/100
40852/40852 [==============================] - 53s 1ms/step - loss: 0.0112 - loss_1: 71.4442 - val_loss: 0.0115 - val_loss_1: 73.0024

Epoch 00038: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-38-73.0024.hdf5
Epoch 39/100
40852/40852 [===========================

40852/40852 [==============================] - 51s 1ms/step - loss: 0.0104 - loss_1: 66.3525 - val_loss: 0.0111 - val_loss_1: 71.2379

Epoch 00069: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-69-71.2379.hdf5
Epoch 70/100
40852/40852 [==============================] - 53s 1ms/step - loss: 0.0103 - loss_1: 65.6313 - val_loss: 0.0111 - val_loss_1: 71.2770

Epoch 00070: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-70-71.2770.hdf5
Epoch 71/100
40852/40852 [==============================] - 52s 1ms/step - loss: 0.0103 - loss_1: 65.7096 - val_loss: 0.0109 - val_loss_1: 69.8202

Epoch 00071: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-71-69.8202.hdf5
Epoch 72/100
40852/40852 [==============================] - 57s 1ms/step - loss: 0.0102 - loss_1: 65.4646 - val_loss: 0.0108 - val_loss_1: 69.2882

Epoch 00072: saving model to ./data/checkpoints/cnn_checkpoints_GDI/weights-72-69.2882.hdf5
Epoch 73/100
40852/40852 [===========================

In [24]:
def undo_scaling(y,target_range,target_min):
    return y*target_range+target_min

In [25]:
weight_files = os.listdir(checkpoint_folder)
weight_files_df = pd.DataFrame(weight_files,columns=['filename'])
weight_files_df['num'] = weight_files_df['filename'].apply(lambda x: int(x.split('-')[1]))
weight_files_df.sort_values(by='num',ascending=True,inplace=True)

In [26]:
#todo: change so we also have a X,y for all_ids
#then for each of 100 epochs, generate the predictions of the model for each epoch
#this dataframe can then be saved, and a file called "get_best_epoch" can be used to generate the best one
#(video_id,target,predictions,dataset)

In [27]:
def predict_and_aggregate(y,X,ids,model,target_col):
    df = pd.DataFrame(y,columns=[target_col])
    side_col = ['L' if i < len(df)/2 else 'R' for i in range(len(df))]
    target_col_pred = target_col + "_pred"
    videoids = [t[0] for t in processed_video_segments if t[0] in ids]
    videoids = videoids + videoids
    df["videoid"] = np.array(videoids)
    df['side'] = side_col
    preds = model.predict(X)
    df[target_col_pred] = undo_scaling(preds[:,0],target_range,target_min)
    df["count"] = 1
    df = df.groupby(['videoid','side'],as_index=False).agg({target_col_pred:np.mean,'count':np.sum,target_col:np.mean})
    df['ones'] = 1
    return df

In [28]:
video_ids = [t[0] for t in processed_video_segments if t[0] in all_ids]
video_ids = video_ids + video_ids
predictions_df = pd.DataFrame(video_ids,columns=['videoid'])
side_col = ['L' if i < len(predictions_df)/2 else 'R' for i in range(len(predictions_df))]
predictions_df[target_col] = y
predictions_df['side'] = side_col
predictions_df = predictions_df.merge(right=datasplit_df[['videoid','dataset']],on=['videoid'],how='left')

In [29]:
for i in range(0,len(weight_files_df)):
    weight_file = weight_files_df['filename'].iloc[i]
    print(weight_file)
    model.load_weights(checkpoint_folder + "/%s" % (weight_file))
    preds = model.predict(X)
    predictions_df["%s_pred_%s" % (target_col,i)] = undo_scaling(preds[:,0],target_range,target_min)

weights-01-111.6367.hdf5
weights-02-90.2141.hdf5
weights-03-94.6918.hdf5
weights-04-96.3010.hdf5
weights-05-84.5516.hdf5
weights-06-83.6618.hdf5
weights-07-81.8188.hdf5
weights-08-82.9676.hdf5
weights-09-77.5708.hdf5
weights-10-75.9001.hdf5
weights-11-74.7404.hdf5
weights-12-78.7979.hdf5
weights-13-76.6790.hdf5
weights-14-77.8526.hdf5
weights-15-73.7235.hdf5
weights-16-86.1161.hdf5
weights-17-79.0839.hdf5
weights-18-72.6741.hdf5
weights-19-84.1615.hdf5
weights-20-70.2833.hdf5
weights-21-73.6044.hdf5
weights-22-75.9847.hdf5
weights-23-71.2925.hdf5
weights-24-73.7759.hdf5
weights-25-73.8977.hdf5
weights-26-73.7183.hdf5
weights-27-83.7829.hdf5
weights-28-74.1485.hdf5
weights-29-70.0612.hdf5
weights-30-68.9415.hdf5
weights-31-73.8631.hdf5
weights-32-74.2919.hdf5
weights-33-69.4494.hdf5
weights-34-71.1034.hdf5
weights-35-72.4519.hdf5
weights-36-69.6521.hdf5
weights-37-78.1076.hdf5
weights-38-73.0024.hdf5
weights-39-71.5186.hdf5
weights-40-70.4331.hdf5
weights-41-72.2568.hdf5
weights-42-74.9

In [30]:
predictions_df.groupby(['videoid','side','dataset'],as_index=False).mean().to_csv("./data/predictions/cnn_%s_predictions_all_epochs.csv" % (target_col),index=False)

FileNotFoundError: [Errno 2] No such file or directory: './data/predictions/cnn_GDI_predictions_all_epochs.csv'